In [ ]:
# Using the trained model for prediction
# Importing necessary libraries
import os
import numpy as np
import pandas as pd
from libraries.utils import get_paths, read_traces, read_json, mapint2var, is_consistent, detection_quality

In [ ]:
# Configuration
CODE = 'theft_protection'               ### application (code) theft_protection, mamba2, lora_ducy
BEHAVIOUR_FAULTY = 'faulty_data'        ### normal, faulty_data
BEHAVIOUR_NORMAL = 'normal'             ### normal, faulty_data
THREAD = 'single'                       ### single, multi
VER = 4                                 ### format of data collection

base_dir = '../../trace_data'              ### can be replaced with 'csv', 'exe_plot', 'histogram'
normalbase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_NORMAL}'
faultybase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_FAULTY}'

print("Normal base path:", normalbase_path)
print("Faulty base path:", faultybase_path)

In [ ]:
train_base_path = os.path.join(normalbase_path, 'train_data')
train_data_path = [os.path.join(train_base_path, x) for x in os.listdir(train_base_path)]
train_varlist_path = [os.path.join(normalbase_path, x) for x in os.listdir(normalbase_path) if 'varlist' in x]

######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(faultybase_path)

train_data_path = [x for x in train_data_path if '.DS_Store' not in x]
train_varlist_path = [x for x in train_varlist_path if '.DS_Store' not in x]
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()
paths_label.sort()

test_data_path = paths_traces
test_label_path = paths_label

print("Train data path:", train_data_path)
print("Train varlist path:", train_varlist_path)
print("Test data path:", test_data_path)
print("Test label path:", test_label_path)

In [ ]:
# Check consistency
if VER == 3 or VER == 4:
    check_con, _ = is_consistent([train_varlist_path[0]] + varlist_path)
    if check_con:
        to_number = read_json(varlist_path[0])
        from_number = mapint2var(to_number)
    else:
        to_number = read_json(train_varlist_path[0])
        from_number = mapint2var(to_number)

sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]

In [ ]:
test_data_path

## Validation

In [ ]:
# prediction for test data
from libraries.anomaly_detection import test_single_id, merge_detections, get_correct_detections
from tensorflow.keras.models import load_model
import joblib

scaler = joblib.load('./scalers/scaler10_v4_id_{}.pkl'.format(CODE))  # Load the trained scaler

## checking the detections against the ground truth
DIFF_VAL = 5 
all_detections = []         # To store detections for each file
y_pred_all = []             # To store the predicted labels
y_true_all = []             # To store the ground truth labels
all_tp = []                 # To store all true positives
all_fp = []                 # To store all false positives
all_fn = []                 # To store all false negatives
all_gt = []                 # To store the ground truth

model = load_model('./trained_models/lstm_v4_id_{}.keras'.format(CODE))  # Load the trained model
sequence_length = 10                                                # Sequence length for the model

# Iterating through each test data file and label file
for test_data, test_label in zip(test_data_path, test_label_path):
    detection, inference_time = test_single_id(test_data, model, sequence_length, scaler)            # Detecting anomalies in the test data
    print("Detection : ", detection)

    print("Detection : ", detection)
    print("len(detection) : ", len(detection))

    merge_detection, agg_ts = merge_detections(detection, diff_val=DIFF_VAL)
    detection = merge_detection

    print("Merge detection : ", merge_detection)
    
    ground_truth_raw = read_traces(test_label)                                               # read ground truth labels from the label file
    ground_truth = ground_truth_raw['labels']                                                # extract labels from dictionary from ground truth data

    label_trace_name = list(ground_truth.keys())[0]
    ground_truth = ground_truth[label_trace_name]

    correct_pred, rest_pred, y_pred, y_true, false_neg = get_correct_detections(merge_detection, ground_truth)  # Comparing detected anomaly with ground truth

    y_pred_all.extend(y_pred)          # predicted labels
    y_true_all.extend(y_true)          # actual ground truth labels

    all_detections.append((test_data, detection, test_label))
    all_tp.append((test_data, correct_pred, test_label))
    all_fp.append((test_data, rest_pred, test_label))
    all_fn.append((test_data, false_neg, test_label))
    all_gt.append((test_data, ground_truth, test_label))

    print("Inference time : ", (inference_time/32))

### Detection Quality after diff_val (percent of non-anomalous trace)

In [ ]:
all_tp_quality = []
all_quality_scores = []
all_gt_count = []
all_tp_count = []
for file_tp in all_detections[0:]:
    print(file_tp)
    file_path = file_tp[0]
    print('File Path:', file_path)
    print('File Name:', os.path.basename(file_path))

    trace = read_traces(file_path)
    # print('Trace Data:', trace)
    time_stamps = [e[1] for e in trace]
    events = [e[0] for e in trace]
    # print('Time Stamps:', time_stamps)
    # print('Events:', events)

    time_stamps = np.array(time_stamps)
    events = np.array(events)

    ### load GT
    label_path = file_tp[2]
    # print('Label Path:', label_path)
    ground_truth_raw = read_traces(label_path)
    # print('Ground Truth Raw:', ground_truth_raw)
    ground_truth = ground_truth_raw['labels']
    # print('Ground Truth:', ground_truth)
    label_trace_name = list(ground_truth.keys())[0]
    ground_truth = ground_truth[label_trace_name]
    # print('Label Trace Name:', label_trace_name)
    # print('Ground Truth Length:', len(ground_truth))
    print('Ground Truth:', ground_truth)

    quality_score = []
    gt_count = 0
    tp_count = 0
    for tp in file_tp[1]:
        print('TP:', tp)
        pd_ts1 = tp[1][0]
        pd_ts2 = tp[1][1]
        # print('Detection Start:', pd_ts1)
        # print('Detection End:', pd_ts2)

        ### get the closest timestamp to pd_ts1 and pd_ts2
        start_tp = np.argmin(np.abs(time_stamps - pd_ts1))
        end_tp = np.argmin(np.abs(time_stamps - pd_ts2))
        print('Start Index:', start_tp)
        print('End Index:', end_tp)
        
        ### get exact timestamp match for pd_ts1 and pd_ts2
        # start_tp = np.where(time_stamps == pd_ts1)[0][0]
        # end_tp = np.where(time_stamps == pd_ts2)[0][0]
        # print('Start Index:', start_index)
        # print('End Index:', end_index)

        ### get the sub trace corresponding to the detection
        det_trace = trace[start_tp:end_tp+1]
        len_det_trace = len(det_trace)
        # print('Detection Trace:', det_trace)
        # print('Detection Trace Length:', len(det_trace))

        ### collect groundtruths that intersect with the detection
        gt_overlap_trace = []
        sel_gt = []
        for gt in ground_truth:
            gt_ts1 = gt[2]
            gt_ts2 = gt[3]
            # print('Ground Truth Start:', gt_ts1)
            # print('Ground Truth End:', gt_ts2)
            cond_1 = pd_ts1 >= gt_ts1 and pd_ts2 <= gt_ts2  ### check if the detection timestamp is within the ground truth timestamp (case 1)
            cond_2 = pd_ts1 <= gt_ts1 and pd_ts2 >= gt_ts2  ### check if the gorund truth timestamp is within the detection timestamp (case 2)
            cond_3 = pd_ts1 >= gt_ts1 and pd_ts1 <= gt_ts2 and pd_ts2 >= gt_ts2    ### partial detection on right of the ground truths, check 5 second difference after this (case 3)
            cond_4 = pd_ts2 <= gt_ts2 and pd_ts2 >= gt_ts1 and pd_ts1 <= gt_ts1   ### partial detection on left of the ground truths, check 5 second difference after this (case 4)

            if cond_1 or cond_2 or cond_3 or cond_4:
                print('GT', gt)
                if gt not in sel_gt:
                    sel_gt.append(gt)

                start_gt = np.where(time_stamps == gt_ts1)[0][0]
                end_gt = np.where(time_stamps == gt_ts2)[0][0]
                # print('Start Index GT:', start_gt)
                # print('End Index GT:', end_gt)
                gt_trace = trace[start_gt:end_gt+1]
                # print('Ground Truth Trace:', gt_trace)
                # print('len of gt:', len(gt_trace))

                
                for (gt_e, gt_t) in gt_trace:
                    if gt_t >= pd_ts1 and gt_t <= pd_ts2:
                        gt_overlap_trace.append((gt_e, gt_t))


        print('Selected Ground Truths:', sel_gt)
        # print('gt overlap_trace:', gt_overlap_trace)

        ### calculate quality of detection (percentge of detection trace that do not overlap with ground truth)
        if len(gt_overlap_trace) == 0:
            # raise ValueError('No ground truth overlap trace found for detection', tp, ', file path:', file_path)
            overlap_percentage = 0.0
        else:
            overlap_percentage = len(gt_overlap_trace) / len_det_trace    ### precision of detection (point-wise)
            overlap_percentage = np.round(overlap_percentage, 2)


            # nonanomaly_percentage = 1 - overlap_percentage
            # nonanomaly_percentage = np.round(nonanomaly_percentage, 2)
            # print('Detection with normal trace (Percentage):', nonanomaly_percentage)
            # quality_score.append(nonanomaly_percentage)
            
        print('Point-wise precision of detection:', overlap_percentage)
        quality_score.append(overlap_percentage)

        
        ### calculate quality metric (detection to ground truth ratio)
        gt_count = len(sel_gt)
        tp_count += 1

        print('')
        # break    

    all_gt_count += [gt_count]
    all_tp_count += [tp_count]
    
    all_tp_quality.append((file_path, quality_score, label_path))
    all_quality_scores.extend(quality_score)
    print('GT count:', gt_count)
    print('TP count:', tp_count)
    print('')   
    # break

print('Total GT count:', all_gt_count)
print('Total TP count:', all_tp_count)

In [ ]:
all_quality_scores = np.array(all_quality_scores)
print('All quality scores:', all_quality_scores)
print('Average quality score:', np.mean(all_quality_scores))
print('Standard deviation of quality scores:', np.std(all_quality_scores))

In [ ]:
all_quality_scores = np.array(all_quality_scores)
print('All quality scores:', all_quality_scores)
print('Average quality score:', np.mean(all_quality_scores))
print('Standard deviation of quality scores:', np.std(all_quality_scores))

In [ ]:
#### Caculate quality score for each file as well as overall quality score
overall_quality_score = []
gtc = 0
tpc = 0
for file_quality, gt_count, tp_count in zip(all_tp_quality, all_gt_count, all_tp_count):
    file_path = file_quality[0]
    quality_score = file_quality[1]
    label_path = file_quality[2]

    print('File Path:', file_path)
    # print('Quality Score:', quality_score)
    # print('Label Path:', label_path)

    if len(quality_score) == 0:
        print('No quality score found for file:', file_path)
        continue

    avg_quality = np.mean(quality_score)
    avg_quality = np.round(avg_quality, 2)
    std_quality = np.std(quality_score)
    std_quality = np.round(std_quality, 2)
    overall_quality_score.extend(quality_score)
    print('Average Quality Score for File:', avg_quality, '±', std_quality)

    ### calculate quality metric (detection to ground truth ratio)
    per_file = tp_count / gt_count
    print('Detection to Ground Truth Ratio for File:', per_file)

    gtc += gt_count
    tpc += tp_count

    print('')

overall_avg = np.mean(overall_quality_score)
overall_std = np.std(overall_quality_score)
overall_avg = np.round(overall_avg, 2)
overall_std = np.round(overall_std, 2)
print('Overall Average Quality Score:', overall_avg, '±', overall_std)
print('')

overall_tgr = tpc / gtc
overall_tgr = np.round(overall_tgr, 2)
print('Overall Detection to Ground Truth Ratio:', overall_tgr)



# print('Total Ground Truth Count:', all_gt_count)
# print('Total True Positive Count:', all_tp_count)
# det2gt = all_tp_count / all_gt_count
# det2gt = np.round(det2gt, 2)
# print('Detection to Ground Truth Ratio:', det2gt)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

y_pred_all = np.array(y_pred_all)
y_true_all = np.array(y_true_all)

# Calculate evaluation metrics
precision = precision_score(y_true_all, y_pred_all)
recall = recall_score(y_true_all, y_pred_all)
f1 = f1_score(y_true_all, y_pred_all)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

conf_matrix = confusion_matrix(y_true_all, y_pred_all)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['Normal', 'Anomaly'])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

### Classwise Detections

In [ ]:
from collections import defaultdict


classwise_fn = defaultdict(list)
classwise_tp = defaultdict(list)
gt_len = 0
for file_fn, file_gt in zip(all_fn, all_gt):
    fn = file_fn[1]
    gt = file_gt[1]
    for label in gt:
        if label in fn:
            classwise_fn[label[4]].append(label)
        else:
            classwise_tp[label[4]].append(label)
            # print('tp:', label)

    gt_len += len(gt)
    # print('file gt:', len(gt))
    # print('file fn:', len(fn))
    # print('\n')
    # break

total_fn = 0
total_tp = 0
keys = set(list(classwise_fn.keys()) + list(classwise_tp.keys()))
# print('keys:', keys)
class_recall = []
for key in keys:
    print('class:', key)
    total_fn += len(classwise_fn[key])
    total_tp += len(classwise_tp[key])

    crecall = len(classwise_tp[key])/(len(classwise_fn[key])+len(classwise_tp[key]))

    # print('not detected:', len(classwise_fn[key]))
    print('detected:', len(classwise_tp[key]))
    print('total anomalies:', len(classwise_fn[key])+len(classwise_tp[key]))
    print('Recall (classwise):', crecall)
    print('\n')

    class_recall.append(crecall)


# print('total fn+tp:', total_fn+total_tp)
# print('total gt:', gt_len)
assert total_fn+total_tp == gt_len, 'total fn+tp not equal to total gt'
print('All class recalls:', class_recall)

## Save Detections

In [ ]:
# Saving the detections for further analysis

######## save detections for the dashboard to plot #############
import traceback
import json
# DIFF_VAL = 0

for test_data, detections, test_label in all_detections:
    # print(test_data, test_label)
    # print(test_label.replace('labels', 'detections'))
    detection_path = test_label.replace('labels', f'lstm_detections')
    detection_path = detection_path.replace('lstm_detections.json', f'lstm_detections_{DIFF_VAL}.json')
    # tp_detection_path = detection_path.replace('ei_detections.json', f'tp_ei_detections_{DIFF_VAL}.json')
    # fp_detection_path = detection_path.replace('ei_detections.json', f'fp_ei_detections_{DIFF_VAL}.json')
    # print(detections)

    detection_dir = os.path.dirname(detection_path)
    # print(detection_dir)
    if not os.path.exists(detection_dir):
        os.makedirs(detection_dir)
        print(f'Created Directory: {detection_dir}')

    try:
        with open(detection_path, 'w') as f:
            json.dump(detections, f)
            print(f'Saved detections in {detection_path}')

            
    except Exception as e:
        traceback.print_exception(e)
        print('Error in saving detections')
        continue

for test_data, detections, test_label in all_tp:
    # print(test_data, test_label)
    # print(test_label.replace('labels', 'detections'))
    detection_path = test_label.replace('labels', 'lstm_detections')
    tp_detection_path = detection_path.replace('lstm_detections.json', f'tp_lstm_detections_{DIFF_VAL}.json')
    # fp_detection_path = detection_path.replace('ei_detections.json', 'fp_ei_detections.json')
    # print(detections)

    detection_dir = os.path.dirname(detection_path)
    # print(detection_dir)
    if not os.path.exists(detection_dir):
        os.makedirs(detection_dir)
        print(f'Created Directory: {detection_dir}')

    try:

        with open(tp_detection_path, 'w') as f:
            json.dump(detections, f)
            print(f'Saved detections in {tp_detection_path}')
            
    except Exception as e:
        traceback.print_exception(e)
        print('Error in saving detections')
        continue

for test_data, detections, test_label in all_fp:
    # print(test_data, test_label)
    # print(test_label.replace('labels', 'detections'))
    detection_path = test_label.replace('labels', 'lstm_detections')
    # tp_detection_path = detection_path.replace('ei_detections.json', 'tp_ei_detections.json')
    fp_detection_path = detection_path.replace('lstm_detections.json', f'fp_lstm_detections_{DIFF_VAL}.json')
    # print(detections)

    detection_dir = os.path.dirname(detection_path)
    # print(detection_dir)
    if not os.path.exists(detection_dir):
        os.makedirs(detection_dir)
        print(f'Created Directory: {detection_dir}')

    try:

        with open(fp_detection_path, 'w') as f:
            json.dump(detections, f)
            print(f'Saved detections in {fp_detection_path}')
            
    except Exception as e:
        traceback.print_exception(e)
        print('Error in saving detections')
        continue

## Diff Val for paper

In [ ]:
#### diff_val for paper

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, average_precision_score, ConfusionMatrixDisplay
import pandas as pd

from libraries.anomaly_detection import test_single_id, merge_detections, get_correct_detections
from tensorflow.keras.models import load_model
import joblib

import traceback
import json

scaler = joblib.load('./scalers/scaler10_v4_id_{}.pkl'.format(CODE))  # Load the trained scaler

model = load_model('./trained_models/lstm_v4_id_{}.keras'.format(CODE))  # Load the trained model
sequence_length = 10                                                # Sequence length for the model

diff_val_res = []  ### used to store results for different diff_val
for DIFF_VAL in range(0, 180):
    if (DIFF_VAL % 5 == 0 and DIFF_VAL <= 50) or DIFF_VAL <= 5:
        print('DIFF_VAL:', DIFF_VAL)

        # DIFF_VAL = 2
        #### Validate model
        all_detections = []  ### format [file1_detection, file2_detection] -> file1_detection: [(state1, state2), (ts1, ts2), filename]
        y_pred_all = []
        y_true_all = []
        all_tp = []
        all_fp = []
        all_fn = []
        all_gt = []

        # tp = 0
        # fp = 0
        # fn = 0
            
        for test_data, test_label in zip(test_data_path, test_label_path):
            print('Testing data:', test_data)
            detection, inference_time = test_single_id(test_data, model, sequence_length, scaler)            # Detecting anomalies in the test data    ### threshold based detection
            # print('Detections:', detection)
            print('len of detections:', len(detection))

            merged_detection, agg_ts = merge_detections(detection, diff_val=DIFF_VAL)
            detection = merged_detection


            ground_truth_raw = read_traces(test_label)
            ground_truth = ground_truth_raw['labels']
            label_trace_name = list(ground_truth.keys())[0]
            ground_truth = ground_truth[label_trace_name]
            # print('ground truths:', ground_truth)
            # print(len(ground_truth))
            print('merged detections', len(merged_detection))

            correct_pred, rest_pred, y_pred, y_true, false_neg = get_correct_detections(merged_detection, ground_truth)  # Comparing detected anomaly with ground truth
            
            y_pred_all.extend(y_pred)
            y_true_all.extend(y_true)

            all_detections += [(test_data, detection, test_label)]  ### used to plot detections
            all_tp += [(test_data, correct_pred, test_label)]
            all_fp += [(test_data, rest_pred, test_label)]
            all_fn += [(test_data, false_neg, test_label)]
            all_gt += [(test_data, ground_truth, test_label)]


        # # Calculate precision
        # precision = precision_score(y_true_all, y_pred_all)
        # print(f'Precision: {precision:.4f}')

        # # Calculate recall
        # recall = recall_score(y_true_all, y_pred_all)
        # print(f'Recall: {recall:.4f}')

        # # Calculate average precision
        # average_precision = average_precision_score(y_true_all, y_pred_all)
        # print(f'Average Precision: {average_precision:.4f}')

        # Calculate F1 score
        f1 = f1_score(y_true_all, y_pred_all)
        # print(f"F1 Score: {f1:.4f}")

        # Calculate confusion matrix
        conf_matrix = confusion_matrix(y_true_all, y_pred_all)
        # print("Confusion Matrix:")

        ### calculate detection quality
        quality_score, total_tp_count, total_gt_count = detection_quality(all_tp, output_score=True)

        ### metric 1 (percent of non-anomalous trace)
        avg_quality_score = np.mean(quality_score)
        std_quality_score = np.std(quality_score)

        print('Detection Quality Results:', quality_score, '±', std_quality_score)

        ### metric 2 (ratio of TP to GT)
        det2gt = total_tp_count / total_gt_count

        f1 = np.round(f1, 2)
        avg_quality_score = np.round(avg_quality_score, 2)
        std_quality_score = np.round(std_quality_score, 2)
        det2gt = np.round(det2gt, 2)

        # print(conf_matrix.shape)
        if conf_matrix.shape[0] == 1:
            print(conf_matrix)
            diff_val_res.append([DIFF_VAL, f1, 0, 0, avg_quality_score, std_quality_score, det2gt])
        else:
            diff_val_res.append([DIFF_VAL, f1, conf_matrix[0][1], conf_matrix[1][0], avg_quality_score, std_quality_score, det2gt])

        
        ################## Saving detections ######################
        for test_data, detections, test_label in all_detections:
            # print(test_data, test_label)
            # print(test_label.replace('labels', 'detections'))
            detection_path = test_label.replace('labels', f'lstm_detections')
            detection_path = detection_path.replace('lstm_detections.json', f'lstm_detections_{DIFF_VAL}.json')
            # tp_detection_path = detection_path.replace('ei_detections.json', f'tp_ei_detections_{DIFF_VAL}.json')
            # fp_detection_path = detection_path.replace('ei_detections.json', f'fp_ei_detections_{DIFF_VAL}.json')
            # print(detections)

            detection_dir = os.path.dirname(detection_path)
            # print(detection_dir)
            if not os.path.exists(detection_dir):
                os.makedirs(detection_dir)
                print(f'Created Directory: {detection_dir}')

            try:
                with open(detection_path, 'w') as f:
                    json.dump(detections, f)
                    print(f'Saved detections in {detection_path}')

                    
            except Exception as e:
                traceback.print_exception(e)
                print('Error in saving detections')
                continue

        for test_data, detections, test_label in all_tp:
            # print(test_data, test_label)
            # print(test_label.replace('labels', 'detections'))
            detection_path = test_label.replace('labels', 'lstm_detections')
            tp_detection_path = detection_path.replace('lstm_detections.json', f'tp_lstm_detections_{DIFF_VAL}.json')
            # fp_detection_path = detection_path.replace('ei_detections.json', 'fp_ei_detections.json')
            # print(detections)

            detection_dir = os.path.dirname(detection_path)
            # print(detection_dir)
            if not os.path.exists(detection_dir):
                os.makedirs(detection_dir)
                print(f'Created Directory: {detection_dir}')

            try:

                with open(tp_detection_path, 'w') as f:
                    json.dump(detections, f)
                    print(f'Saved detections in {tp_detection_path}')
                    
            except Exception as e:
                traceback.print_exception(e)
                print('Error in saving detections')
                continue

        for test_data, detections, test_label in all_fp:
            # print(test_data, test_label)
            # print(test_label.replace('labels', 'detections'))
            detection_path = test_label.replace('labels', 'lstm_detections')
            # tp_detection_path = detection_path.replace('ei_detections.json', 'tp_ei_detections.json')
            fp_detection_path = detection_path.replace('lstm_detections.json', f'fp_lstm_detections_{DIFF_VAL}.json')
            # print(detections)

            detection_dir = os.path.dirname(detection_path)
            # print(detection_dir)
            if not os.path.exists(detection_dir):
                os.makedirs(detection_dir)
                print(f'Created Directory: {detection_dir}')

            try:

                with open(fp_detection_path, 'w') as f:
                    json.dump(detections, f)
                    print(f'Saved detections in {fp_detection_path}')
                    
            except Exception as e:
                traceback.print_exception(e)
                print('Error in saving detections')
                continue
        ################## Saving detections ######################

        
        
        # break

df = pd.DataFrame(diff_val_res, columns=['diff_val', 'f1_score', 'FP', 'FN', 'pdPrecision', 'pdPrecStd', 'tp_gt_ratio'], index=None)
###### save df to excel
df.to_excel(f'../../results/diff_val/{CODE}_lstm_results.xlsx', index=False)
print(df)